In [ ]:
import polars as pl
import pandas as pd
import rtsvg
rt = rtsvg.RACETrack()
_node_colors_ = {'a':'yellow', 'b':'blue', 'c':'yellow', 'd':'blue'}
df = pl.DataFrame({'fm':'a b c d a e f g'.split(),
                   'to':'b c d a d f g e'.split(),
                   'gp':'a a a b b c c c'.split()})
rt.co_mgr.str_to_color_lu['a'] = '#ff0000'
rt.co_mgr.str_to_color_lu['b'] = '#00ff00'
_relates_   = [('fm','to')]
_pos_       = {'a':[0,0],'b':[1,0],'c':[1,1],'d':[0,1], 'g': (0.25, 0.68), 'e': (0.50, 0.35), 'f': (0.75, 0.68)}
_ln_params_ = {'relationships':_relates_, 'pos':_pos_, 'color_by':'gp', 'node_color':_node_colors_, 'draw_labels':True}
_l_  = rt.link    (df, track_state=True, **_ln_params_)
_ln_ = rt.linkNode(df, track_state=True, **_ln_params_)
_l_._repr_svg_(), _ln_._repr_svg_()
#rt.tile([_l_, _ln_], spacer=10)
_igl_       = rt.interactiveGraphLayout(df, _ln_params_, w=300, h=300)
_igl_